In [1]:
import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import time
from datetime import datetime, timedelta
from EV_class import EV_BATTERY

In [7]:
print(datetime(2024, 6, 19, 18, 0))
a = datetime(2024, 6, 19, 18, 0)
b = datetime(2024, 6, 20, 7, 30)


tuple_time = (a, b)
c = b - a 
d = tuple_time[1] - tuple_time[0]

2024-06-19 18:00:00


In [3]:
print(int(c.total_seconds()/(60*10)))
print(type(c))

3
<class 'datetime.timedelta'>


In [8]:
Batt = EV_BATTERY(P_charg=46, capacity=62, P_dischar=7, eta = 0.8, n_EV=3)

Batt.read_spot_price('Price_data/bourseEpex_06_06_2024.csv', resample=True)
Batt.reformate_charing_time(tuple_time)
matrix = Batt.generate_contiguous_matrix(SOC_random=False,
                                         SOC_ini=0.5*np.ones((3,)),
                                         SOC_final=0.95*np.ones((3,)))
charge, discharge, SOC = Batt.battery_dispatch('VPP')
total_price= Batt.obj_value

df = Batt.generate_result_df()
print(df)

Not random
Not random
Not random
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1299 rows, 1728 columns and 3072 nonzeros
Model fingerprint: 0xd6909477
Model has 864 quadratic constraints
Variable types: 1296 continuous, 432 integer (432 binary)
Coefficient statistics:
  Matrix range     [2e-01, 6e+01]
  QMatrix range    [4e+02, 4e+02]
  QLMatrix range   [1e+00, 4e+02]
  Objective range  [2e-03, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 5e+01]
  QRHS range       [4e+02, 4e+02]
Presolve removed 997 rows and 1422 columns
Presolve time: 0.02s
Presolved: 446 rows, 354 columns, 1081 nonzeros
Variable types: 273 continuous, 81 integer (81 binary)
Found heuristic solution: objective 0.2245270

Root relaxation: objective 1.460949e+00, 78 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective 

In [11]:
fig, ax = plt.subplots(figsize = (5,3), dpi = 300)

#ls_time = df['Time']

ls_time = [i for i in range(len(Batt.price_ls))]
#ax2 = ax.twinx()


#ax2.plot(ls_time, SOC[:,0], label = 'VPP', color ='red')
ax.plot(ls_time, Batt.price_ls, label = 'Price', linestyle='--')
ax.legend()
ax.set_ylabel('Price [$/MWh]')
#ax2.set_ylabel('State of charge')
#ax2.legend(loc = 'lower right')



Text(0, 0.5, 'Price [$/MWh]')

In [12]:
print(SOC)

[[0.5        0.5        0.5       ]
 [0.4811828  0.4811828  0.4811828 ]
 [0.46236559 0.46236559 0.46236559]
 [0.44354839 0.44354839 0.44354839]
 [0.42473118 0.42473118 0.42473118]
 [0.40591398 0.40591398 0.40591398]
 [0.38709677 0.38709677 0.38709677]
 [0.36827957 0.36827957 0.36827957]
 [0.34946237 0.34946237 0.34946237]
 [0.33064516 0.33064516 0.33064516]
 [0.31182796 0.31182796 0.31182796]
 [0.29301075 0.29301075 0.29301075]
 [0.27419355 0.27419355 0.27419355]
 [0.25537634 0.25537634 0.25537634]
 [0.23655914 0.23655914 0.23655914]
 [0.21774194 0.21774194 0.21774194]
 [0.19892473 0.19892473 0.19892473]
 [0.18010753 0.18010753 0.18010753]
 [0.16129032 0.16129032 0.16129032]
 [0.16129032 0.16129032 0.16129032]
 [0.16129032 0.16129032 0.16129032]
 [0.16129032 0.16129032 0.16129032]
 [0.16129032 0.16129032 0.16129032]
 [0.16129032 0.16129032 0.16129032]
 [0.16129032 0.16129032 0.16129032]
 [0.28494624 0.16129032 0.28494624]
 [0.40860215 0.19354839 0.40860215]
 [0.40860215 0.3172043  0.40

In [6]:
import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import time
from datetime import datetime, timedelta
from EV_class import EV_BATTERY
import plotly.express as px

# Set the page layout
st.set_page_config(layout="wide")

# Sidebar for input sliders
st.sidebar.title("Input Sliders")

slider_values = []
for i in range(1, 6):
    value = st.sidebar.slider(f'Slider {i}', 0, 100, 50)
    slider_values.append(value)


# State of charge data
SOC_ini = st.sidebar.slider(f'Initial State of Charge', 0, 100, 50)
SOC_final = st.sidebar.slider(f'Desired State of Charge', 0, 100, 95)
    

start_time = datetime.now().replace(hour=18, minute=0, second=0, microsecond=0)
end_time = start_time + timedelta(days=1, hours=-1)

# Create the slider charging schedule timeline
charging_schedule = st.sidebar.slider(
    "Schedule your appointment:",
    min_value=start_time,
    max_value=end_time,
    value=(start_time + timedelta(hours=0, minutes=0), start_time + timedelta(hours=5, minutes=45)),
    step=timedelta(minutes=10),
    format="MM/DD HH:mm"
)
st.write("You're scheduled for:", charging_schedule)
print(charging_schedule)

#=====================================================================================
#============= Run simulation ========================================================
#=====================================================================================

Batt = EV_BATTERY(P_charg=46, capacity=62, P_dischar=7, eta = 0.8, n_EV=3)

Batt.read_spot_price('Price_data/bourseEpex_06_06_2024.csv', resample=True)
Batt.reformate_charing_time(charging_schedule)

print(Batt.start_pos)
print(Batt.len_ones)
Batt.generate_contiguous_matrix(SOC_random=False,
                                         SOC_ini= SOC_ini/100*np.ones((3,)),
                                         SOC_final= SOC_final/100*np.ones((3,)))
charge, discharge, SOC = Batt.battery_dispatch('VPP')
total_price= Batt.obj_value

df = Batt.generate_result_df()

# Main dashboard layout
col1, col2 = st.columns(2)

# Top-left: Text and number
col1.subheader("Charging operation for Nissan LEAF")
col1.metric(label="Charging cost", value=SOC_ini)
col1.metric(label="Charging cost", value=SOC_ini)

# Top-right: Bar plot
col2.subheader("Bar Plot")
fig, ax = plt.subplots()
ax.bar(range(1, 6), slider_values)
ax.set_xlabel("Slider")
ax.set_ylabel("Value")
col2.pyplot(fig)

# Bottom: Line plot
st.subheader("Line Plot")


fig = px.scatter(df, x="Time", y="Price [$/MWh]")

event = st.plotly_chart(fig, on_select="rerun")


(datetime.datetime(2024, 6, 19, 18, 0), datetime.datetime(2024, 6, 19, 23, 45))
0
34
Not random
Not random
Not random
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1299 rows, 1728 columns and 2790 nonzeros
Model fingerprint: 0xfe56d12c
Model has 864 quadratic constraints
Variable types: 1296 continuous, 432 integer (432 binary)
Coefficient statistics:
  Matrix range     [2e-01, 6e+01]
  QMatrix range    [4e+02, 4e+02]
  QLMatrix range   [1e+00, 4e+02]
  Objective range  [2e-03, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 5e+01]
  QRHS range       [4e+02, 4e+02]
Presolve removed 1186 rows and 1611 columns
Presolve time: 0.01s
Presolved: 116 rows, 118 columns, 329 nonzeros
Variable types: 84 continuous, 34 integer (34 binary)
Found heuristic solution: objective -2.5945456

Root relaxation: objective -2.174294e+00, 41 iteratio

In [7]:
df.head()

,0,1,2,Price [$/MWh],Time
0,0.500000,0.500000,0.500000,84.92,2024-06-19 18:00:00
1,0.481183,0.481183,0.481183,84.92,2024-06-19 18:10:00
2,0.462366,0.462366,0.462366,84.92,2024-06-19 18:20:00
3,0.443548,0.443548,0.443548,84.92,2024-06-19 18:30:00
4,0.424731,0.424731,0.424731,84.92,2024-06-19 18:40:00


In [9]:
import plotly.express as px
data_canada = pd.DataFrame({'year' : [1, 2,3], 'pop': [3, 10, 20]})
fig = px.bar(data_canada, x='year', y='pop')
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [2]:
import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import time
from datetime import datetime, timedelta
from EV_class import EV_BATTERY
import plotly.express as px
import plotly.graph_objects as go

# Set the page layout
st.set_page_config(layout="wide")

# Sidebar for input sliders
st.sidebar.title("User Input")

charge_option = st.sidebar.selectbox(
    "Select your charger",
    ("Electra", "Wallbox", "Circontrol"))

charging_type = st.sidebar.multiselect(
    "Charging type",
    ["V2G", "Smart", "Fast"])

# State of charge data
SOC_ini = st.sidebar.slider(f'Initial State of Charge', 0, 100, 50)
SOC_final = st.sidebar.slider(f'Desired State of Charge', 0, 100, 95)
    

#start_time = datetime.now().replace(hour=18, minute=0, second=0, microsecond=0)
start_time = datetime(2024, 6, 19, 18, 00)
end_time = start_time + timedelta(days=1, hours=-1)

# Create the slider charging schedule timeline
charging_schedule = st.sidebar.slider(
    "Schedule your charging schedule:",
    min_value=start_time,
    max_value=end_time,
    value=(start_time + timedelta(hours=0, minutes=0), start_time + timedelta(hours=5, minutes=40)),
    step=timedelta(minutes=10),
    format="MM/DD HH:mm"
)

#=====================================================================================
#============= Run simulation ========================================================
#=====================================================================================

Batt = EV_BATTERY(P_charg=46, capacity=62, P_dischar=7, eta = 0.8, n_EV=3)

Batt.read_spot_price('Price_data/bourseEpex_06_06_2024.csv', resample=True)
Batt.reformate_charing_time(charging_schedule)
Batt.definec_charger_charac(charge_type=charge_option)

print(Batt.start_pos)
print(Batt.len_ones)
Batt.generate_contiguous_matrix(SOC_random=False,
                                         SOC_ini= SOC_ini/100*np.ones((3,)),
                                         SOC_final= SOC_final/100*np.ones((3,)))
charge, discharge, SOC = Batt.battery_dispatch('VPP')
# Generate the df only for the VPP
df_VPP = Batt.generate_result_df()

total_price= Batt.obj_value

#Look at the other benchmark
total_price_ls = []
charge, discharge, SOC = Batt.battery_dispatch('VPP')
total_price_ls.append(Batt.obj_value)
charge_smart, discharge_smart, SOC_smart = Batt.battery_dispatch('smart')
total_price_ls.append(Batt.obj_value)
df_Smart = Batt.generate_result_df()
charge_fast, discharge_fast, SOC_fast = Batt.battery_dispatch('fast')
total_price_ls.append(Batt.obj_value)
df_fast = Batt.generate_result_df()

#Compute the electra price
price_electra = 0.49*62*(SOC_final - SOC_ini)/100
total_price_ls.append(price_electra)

dict_df = {'V2G': df_VPP, 'Smart': df_Smart, 
           'Fast': df_fast}

# Main dashboard layout
col1, col2 = st.columns(2)

# Top-left: Text and number
col1.subheader("Charging operation for Nissan LEAF")
col1.metric(label="Charging cost [$]", value=np.round(total_price,2))
col1.metric(label="Initial SOC [%]", value=SOC_ini)
col1.metric(label="Desired SOC [%]", value=SOC_final)

# Top-right: Bar plot
col2.subheader("Comparison of the charging method")
#fig, ax = plt.subplots(figsize = (4,3), dpi = 300)
#bars = ax.bar(['VPP', 'Smart', 'Fast'], total_price_ls)
#ax.set_ylim(np.min(total_price_ls)*1.15, np.max(total_price_ls)*1.15)
# Adding annotations
#for bar, value in zip(bars, total_price_ls):
  #  height = np.round(bar.get_height(),2)
  #  print(height)
 #   ax.text(bar.get_x() + bar.get_width() / 2.0, height, f'{np.round(value,2)}', ha='center', va='bottom')
#ax.set_ylabel('Price [$]')


data_price = pd.DataFrame({'Price [$]' : total_price_ls, 'Method': ['V2G', 'Smart', 'Fast', 'Electra']})
fig = px.bar(data_price, x='Method', y='Price [$]')

#col2.pyplot(fig)
col2.plotly_chart(fig)

# Bottom: Line plot
st.subheader("Charging profile")


# Create figure
fig = go.Figure()

# Add scatter plot for 'Price [$/MWh]'
fig.add_trace(
    go.Scatter(x=df_VPP['Time'], y=df_VPP['Price [$/MWh]'], name="Price [$/MWh]", mode='lines+markers', yaxis="y1")
)


for elem in charging_type:
    print(elem) 
    print(charge_option)
    df = dict_df[elem]
    # Add scatter plot for 'SOC'
    fig.add_trace(
        go.Scatter(x=df['Time'], y=df['SOC 0'], name=f"SOC {elem}", mode='lines+markers', yaxis="y2")
    )



# Create axis objects
fig.update_layout(
    xaxis=dict(
        title="Time"
    ),
    yaxis=dict(
        title="Price [$/MWh]",
        titlefont=dict(color="#1f77b4"),
        tickfont=dict(color="#1f77b4")
    ),
    yaxis2=dict(
        title="SOC",
        titlefont=dict(color="#ff7f0e"),
        tickfont=dict(color="#ff7f0e"),
        anchor="x",
        overlaying="y",
        side="right"
    )
)

# Update layout
fig.update_layout(
    legend=dict(x=0, y=1),
    margin=dict(l=40, r=40, t=40, b=40)
)

event = st.plotly_chart(fig)


0
34
Not random
Not random
Not random
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2163 rows, 1728 columns and 3654 nonzeros
Model fingerprint: 0x8c615967
Model has 864 quadratic constraints
Variable types: 1296 continuous, 432 integer (432 binary)
Coefficient statistics:
  Matrix range     [2e-01, 6e+01]
  QMatrix range    [4e+02, 4e+02]
  QLMatrix range   [1e+00, 4e+02]
  Objective range  [2e-03, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 2e+02]
  QRHS range       [4e+02, 4e+02]
Presolve removed 2052 rows and 1612 columns
Presolve time: 0.01s
Presolved: 114 rows, 117 columns, 322 nonzeros
Variable types: 83 continuous, 34 integer (34 binary)
Found heuristic solution: objective 2.5945456

Root relaxation: objective 2.174294e+00, 37 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Object

In [3]:
print(60*1/4)

15.0


In [2]:
import pandas as pd

df_aFRR_price = pd.read_csv('Price_data/aFRR_06_06_2024.csv')

# Split the column into two columns
df_aFRR_price[['Start Time', 'End Time']] = df_aFRR_price['ISP CET/CEST'].str.split(' - ', expand=True)

print(df_aFRR_price.head())
# Convert to datetime
df_aFRR_price['Start Time'] = pd.to_datetime(df_aFRR_price['Start Time'], format='%d/%m/%Y %H:%M:%S')
df_aFRR_price['End Time'] = pd.to_datetime(df_aFRR_price['End Time'], format='%d/%m/%Y %H:%M:%S')


# Define the start and end datetime for the range
start_date = pd.to_datetime('06/06/2024 18:00', format='%d/%m/%Y %H:%M')
end_date = pd.to_datetime('07/07/2024 18:00', format='%d/%m/%Y %H:%M')


# Filter the dataframe to keep rows within the date range
filtered_df = df_aFRR_price[(df_aFRR_price['Start Time'] >= start_date) & (df_aFRR_price['Start Time'] <= end_date)]


filtered_df['Price Up (EUR/MWh)'] = filtered_df['Price Up (EUR/MWh)'].replace(' ', '0')
filtered_df['Price Up (EUR/MWh)'] = filtered_df['Price Up (EUR/MWh)'].replace('', '0')
filtered_df['Price Down (EUR/MWh)'] = filtered_df['Price Down (EUR/MWh)'].replace(' ','0')
filtered_df['Price Down (EUR/MWh)'] = filtered_df['Price Down (EUR/MWh)'].replace('','0')

filtered_df['Price Up (EUR/MWh)'] = filtered_df['Price Up (EUR/MWh)'].astype(float)
filtered_df['Price Down (EUR/MWh)'] = filtered_df['Price Down (EUR/MWh)'].astype(float)

for elem in filtered_df['Price Up (EUR/MWh)'].values:
    print(elem)

filtered_df.head()

                                ISP CET/CEST    Area Reserve Type  \
0  06/06/2024 00:00:00 - 06/06/2024 00:15:00  SCA|FR      aFRR LS   
1  06/06/2024 00:15:00 - 06/06/2024 00:30:00  SCA|FR      aFRR LS   
2  06/06/2024 00:30:00 - 06/06/2024 00:45:00  SCA|FR      aFRR LS   
3  06/06/2024 00:45:00 - 06/06/2024 01:00:00  SCA|FR      aFRR LS   
4  06/06/2024 01:00:00 - 06/06/2024 01:15:00  SCA|FR      aFRR LS   

          Source Type of Product Price Type Price Up (EUR/MWh)  \
0  Not Specified        Standard    AVERAGE              47.41   
1  Not Specified        Standard    AVERAGE              32.52   
2  Not Specified        Standard    AVERAGE              18.06   
3  Not Specified        Standard    AVERAGE                      
4  Not Specified        Standard    AVERAGE              14.54   

  Price Down (EUR/MWh)           Start Time             End Time  
0                31.86  06/06/2024 00:00:00  06/06/2024 00:15:00  
1                       06/06/2024 00:15:00  06/06/202

/var/folders/sc/srskfqx51lg6fw73v_l70gtc0000gn/T/ipykernel_31044/4065863029.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Price Up (EUR/MWh)'] = filtered_df['Price Up (EUR/MWh)'].replace(' ', '0')
/var/folders/sc/srskfqx51lg6fw73v_l70gtc0000gn/T/ipykernel_31044/4065863029.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Price Up (EUR/MWh)'] = filtered_df['Price Up (EUR/MWh)'].replace('', '0')
/var/folders/sc/srskfqx51lg6fw73v_l70gtc0000gn/T/ipykernel_31044/4065863029.py:

,ISP CET/CEST,Area,Reserve Type,Source,Type of Product,Price Type,Price Up (EUR/MWh),Price Down (EUR/MWh),Start Time,End Time
72,06/06/2024 18:00:00 - 06/06/2024 18:15:00,SCA|FR,aFRR LS,Not Specified,Standard,AVERAGE,-78.20,20.53,2024-06-06 18:00:00,2024-06-06 18:15:00
73,06/06/2024 18:15:00 - 06/06/2024 18:30:00,SCA|FR,aFRR LS,Not Specified,Standard,AVERAGE,0.00,36.01,2024-06-06 18:15:00,2024-06-06 18:30:00
74,06/06/2024 18:30:00 - 06/06/2024 18:45:00,SCA|FR,aFRR LS,Not Specified,Standard,AVERAGE,-7.68,38.91,2024-06-06 18:30:00,2024-06-06 18:45:00
75,06/06/2024 18:45:00 - 06/06/2024 19:00:00,SCA|FR,aFRR LS,Not Specified,Standard,AVERAGE,34.97,56.45,2024-06-06 18:45:00,2024-06-06 19:00:00
76,06/06/2024 19:00:00 - 06/06/2024 19:15:00,SCA|FR,aFRR LS,Not Specified,Standard,AVERAGE,13.32,34.76,2024-06-06 19:00:00,2024-06-06 19:15:00


In [4]:
print(type(filtered_df['Price Up (EUR/MWh)'][0]))
print(filtered_df['Price Up (EUR/MWh)'][0])


<class 'str'>
 


# Look at the aFFR bids additionnaly

In [1]:
from EV_class import EV_BATTERY
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
# Define the start and end datetime for the range
date_range = [pd.to_datetime('06/06/2024 18:00', format='%d/%m/%Y %H:%M'), 
              pd.to_datetime('07/07/2024 18:00', format='%d/%m/%Y %H:%M')]

SOC_ini = 30
SOC_final = 100

charging_schedule = [datetime(2024, 6, 19, 18, 00), datetime(2024, 6, 20, 8, 00)]

n_EV = 10

Batt = EV_BATTERY(P_charg=46, capacity=62, P_dischar=7, eta = 0.8, n_EV=n_EV)

Batt.read_spot_price('Price_data/bourseEpex_06_06_2024.csv', resample=True, sampling = 4)
Batt.read_aFFR_price('Price_data/aFRR_06_06_2024.csv', date_range=date_range)
Batt.reformate_charing_time(charging_schedule)
Batt.definec_charger_charac(charge_type='Wallbox')

print(Batt.start_pos)
print(Batt.len_ones)
Batt.generate_contiguous_matrix(SOC_random=False,
                                         SOC_ini= SOC_ini/100*np.ones((n_EV,)),
                                         SOC_final= SOC_final/100*np.ones((n_EV,)))
charge, discharge, SOC = Batt.battery_dispatch('aFRR')


-78.2
0.0
-7.68
34.97
13.32
-3.62
7.1
12.63
12.64
0.0
0.0
0.0
37.53
23.98
14.92
0.0
16.11
0.0
16.11
16.11
11.97
11.97
11.97
0.0
0.0
0.0
0.0
0.0
11.43
0.0
0.0
0.0
56.34
9.55
9.36
0.0
0.0
0.0
0.0
0.0
0.0
0.0
15.09
15.09
15.09
15.09
15.09
15.09
0.0
14.2
14.2
16.9
0.0
16.9
24.08
38.39
162.55
46.0
44.29
15.43
103.69
27.64
13.59
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
8.2
6.66
-4.05
-7.68
-16.96
-16.96
1.86
-15.76
97.17
109.51
41.95
2.97
0.0
-17.58
-15.38
0.0
0.0
0.0
0.0
0.0
0.0
-18.55
7.53
20.13
10.86
0.0
0.0
0.0
13.38
13.38
0.0
0.0
15.31
15.31
16.89
16.89
16.9
16.89
16.9
16.9
26.45
13.39
13.39
13.38
-78.2
0.0
-7.68
34.97
13.32
-3.62
7.1
12.63
12.64
0.0
0.0
0.0
37.53
23.98
14.92
0.0
16.11
0.0
16.11
16.11
11.97
11.97
11.97
0.0
0.0
0.0
0.0
0.0
11.43
0.0
0.0
0.0
56.34
9.55
9.36
0.0
0.0
0.0
0.0
0.0
0.0
0.0
15.09
15.09
15.09
15.09
15.09
15.09
0.0
14.2
14.2
16.9
0.0
16.9
24.08
38.39
162.55
46.0
44.29
15.43
103.69
27.64
13.59
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
8.2
6.66

/Users/adrienguenard/Desktop/EF/1.WIP/V2Gopt/EVagr/EVagreg/EV_class.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Price Up (EUR/MWh)'] = filtered_df['Price Up (EUR/MWh)'].replace(' ', '0')
/Users/adrienguenard/Desktop/EF/1.WIP/V2Gopt/EVagr/EVagreg/EV_class.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Price Up (EUR/MWh)'] = filtered_df['Price Up (EUR/MWh)'].replace('', '0')
/Users/adrienguenard/Desktop/EF/1.WIP/V2Gopt/EVagr/EVagreg/EV_class.py:84: SettingWithCopyWarni

Found heuristic solution: objective -12.4816575
Found heuristic solution: objective -11.1244200

Root relaxation: objective 4.026206e+01, 1388 iterations, 0.01 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   40.26206    0   15  -11.12442   40.26206   462%     -    0s
H    0     0                      -9.8096365   40.26206   510%     -    0s
H    0     0                      39.8702199   40.26206  0.98%     -    0s
     0     0   40.21486    0   18   39.87022   40.21486  0.86%     -    0s
H    0     0                      39.9403922   40.21486  0.69%     -    0s
     0     0   40.19857    0   29   39.94039   40.19857  0.65%     -    0s
H    0     0                      40.0043597   40.19857  0.49%     -    0s
     0     0   40.19471    0   24   40.00436   40.19471  0.48%     -    0s
     0     0   40.19470    0   26   40.00436   40.19470  0.48%

In [2]:
print(Batt.bid_up_ls)
print(Batt.bid_down_ls)

[ 0.    0.    0.   70.   70.    0.    0.   70.    0.    0.    0.    0.
 70.   67.25  0.    0.    0.    0.    0.    0.    0.   70.   70.    0.
  0.    0.    0.    0.   70.    0.    0.    0.   70.   70.   70.    0.
  0.    0.    0.    0.    0.    0.   48.75  0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
[  0.    0.    0.    0.    0.    0.  110.    0.  220.    0.    0.    0.
   0.    0.  220.  220.  220.    0.  220.  220.  220.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.  154.    0.    0.  220.  220.  220.  220.  220.
   0.  220.  220.  220.    0.  157.8 110.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    